In [4]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import RMSprop
from torch.utils.data import random_split, DataLoader, TensorDataset
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [6]:
features,target = make_classification(n_classes=2,
                                      n_samples=1000,
                                      n_features=10)
features_train,features_test,target_train,target_test = train_test_split(features,target,test_size=0.1,random_state=1)

torch.manual_seed(0)
np.random.seed(0)

x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1,1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1,1)

In [7]:
class SimpleNeuralNet(nn.Module):
  def __init__(self,layer_size_1=10,layer_size_2=10):
    super(SimpleNeuralNet,self).__init__()
    self.sequential = torch.nn.Sequential(
        torch.nn.Linear(10,layer_size_1),
        torch.nn.ReLU(),
        torch.nn.Linear(layer_size_1,layer_size_2),
        torch.nn.ReLU(),
        torch.nn.Linear(layer_size_2,1),
        torch.nn.Sigmoid()
    )
  def forward(self,x):
    x = self.sequential(x)
    return x

In [8]:
config = {
    "layer_size_1" : tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "layer_size_2" : tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "lr" : tune.loguniform(1e-4, 1e-1),
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=1000,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    parameter_columns=["layer_size_1", "layer_size_2", "lr"],
    metric_columns=["loss"]
)

In [13]:
def train_model(config,epochs=3):
  network = SimpleNeuralNet(config['layer_size_1'],config['layer_size_2'])

  criterion = nn.BCELoss()
  optimizer = optim.Adam(network.parameters(),lr=config['lr'])

  train_data = TensorDataset(x_train,y_train)
  train_loader = DataLoader(train_data,batch_size=100,shuffle=True)

  network = torch.compile(network)

  for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
      optimizer.zero_grad()
      outputs = network(data)
      loss = criterion(outputs,target)
      loss.backward()
      optimizer.step()
      tune.report({'loss': loss.item()})

In [14]:
result = tune.run(
    train_model,
    resources_per_trial={"cpu": 2},
    config=config,
    num_samples=1,
    scheduler=scheduler,
    progress_reporter=reporter
)

best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config:" .format(best_trial.config))
print("Best trial final validation loss:".format (best_trial.last_result['loss']))

best_trained_model = SimpleNeuralNet(best_trial.config['layer_size_1'],best_trial.config['layer_size_2'])

2025-11-12 04:45:20,724	WARNING callback.py:143 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2025-11-12 04:45:20,727	WARNING tune.py:902 -- AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


+--------------------------------------------------------------------+
| Configuration for experiment     train_model_2025-11-12_04-45-20   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        AsyncHyperBandScheduler           |
| Number of trials                 1                                 |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_2025-11-12_04-45-20

Trial status: 1 PENDING
Current time: 2025-11-12 04:45:20. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+-------------------------------------------------+
| Trial name                status             lr |
+-------------------------------------------------+
| train_model_64db5_00000   PENDING    0.00205476 |
+-------------------------------------------------+

Trial train_model_64db5_00000 started

(train_model pid=8368) W1112 04:45:44.382000 8368 torch/utils/cpp_extension.py:118] [0/0] No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'



Trial status: 1 RUNNING
Current time: 2025-11-12 04:45:50. Total running time: 30s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
+-------------------------------------------------+
| Trial name                status             lr |
+-------------------------------------------------+
| train_model_64db5_00000   RUNNING    0.00205476 |
+-------------------------------------------------+

Trial train_model_64db5_00000 finished iteration 1 at 2025-11-12 04:45:51. Total running time: 30s
+--------------------------------------------------+
| Trial train_model_64db5_00000 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                         17.8305 |
| time_total_s                             17.8305 |
| training_iteration                             1 |
| loss                                     0.68308 |
+--------------------------------------------------+

Trial train_model_64db5_00000 fin

2025-11-12 04:45:51,958	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_2025-11-12_04-45-20' in 0.0063s.



Trial train_model_64db5_00000 finished iteration 27 at 2025-11-12 04:45:51. Total running time: 31s
+--------------------------------------------------+
| Trial train_model_64db5_00000 result             |
+--------------------------------------------------+
| checkpoint_dir_name                              |
| time_this_iter_s                         0.00944 |
| time_total_s                              17.947 |
| training_iteration                            27 |
| loss                                     0.42595 |
+--------------------------------------------------+

Trial train_model_64db5_00000 completed after 27 iterations at 2025-11-12 04:45:51. Total running time: 31s

Trial status: 1 TERMINATED
Current time: 2025-11-12 04:45:51. Total running time: 31s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
+------------------------------------------------------------------------------------------+
| Trial name                status               lr     iter     total time (s)       lo